In [2]:
import pandas as pd
import os
import pickle
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from matplotlib import pylab as plt
import re
import altair as alt
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS = STOP_WORDS.union({'cup','tsp','cups','tbs','teaspoon','tablespoon'})

In [10]:
epi = os.listdir('static')
epi = [x for x in epi if 'all' in x]
for i,filn in enumerate(epi):
    if i ==0:
        df = pd.read_json('static/'+filn)
    else:
        df = df.append(pd.read_json('static/'+filn),ignore_index = True)

df['Number of Ratings'] = df['Number of Ratings'].apply(lambda x: int(str(x).replace('Ratings','')))
def return_length(x):
    if type(x) != type([]) and type(x) != type(1.0):
        x = x.split()
        x = len(x)
        return x
    elif type(x) == type([]):
        return len(' '.join(x))
    else:
        return None
def return_join(x):
    if type(x) == type([]):
        return ' '.join(x).lower()
    else:
        return x.lower()
def return_length_list(x):
    if type(x) ==str:
        return 1
    elif type(x) == list:
        return len(x)
    else:
        return None
        
    
df['lname'] = df['Recipe Name'].apply(return_length)
df['ldesc'] = df['Description'].apply(return_length)
df['lingredients'] = df['Ingredients'].apply(return_length_list)
df['linstructions'] = df.Instructions.apply(return_length_list)
df = df.dropna()
df.Ingredients = df.Ingredients.apply(lambda x: return_join(x))
df.Description = df.Description.apply(lambda x: return_join(x))
df['Recipe Name'] = df['Recipe Name'].apply(lambda x: return_join(x))
df.Instructions = df.Instructions.apply(lambda x: return_join(x))

In [341]:
tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(1,1),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Recipe Name'])
out = tfidf.transform(df['Recipe Name']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Single Words','TFIDF Score']
chart1 = alt.Chart(values).mark_bar().encode(x=alt.X('TFIDF Score'),y=alt.Y('Single Words',sort=None))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(2,2),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Recipe Name'])
out = tfidf.transform(df['Recipe Name']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Bigrams','TFIDF Score']
chart2 = alt.Chart(values).mark_bar().encode(y=alt.Y('Bigrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(3,3),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Recipe Name'])
out = tfidf.transform(df['Recipe Name']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Trigrams','TFIDF Score']
chart3 = alt.Chart(values).mark_bar().encode(y=alt.Y('Trigrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(4,4),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Recipe Name'])
out = tfidf.transform(df['Recipe Name']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Quadgrams','TFIDF Score']
chart4 = alt.Chart(values).mark_bar().encode(y=alt.Y('Quadgrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

alt.vconcat((chart1 | chart2) , (chart3 | chart4)).configure_axis( labelFontSize=12).save('templates/ngram.html')
with open('templates/ngram.html','r') as f:
    lines = f.readlines()
with open('templates/ngram.html','w') as w:
    w.write('{% extends "layout.html" %}\n{% block title %}Home{% endblock %}\n{% block content %}')
    w.write('\n')
    w.write('<br><br><br>')
    w.write('<body class="starter-template text-center py-2">')
    w.write('Utilizing natural language processing we can investigate common n-grams in the recipe name across all recipes.  The term-frequency times the inverse document-frequency (tf-idf) is calculated in scikit learn using their <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">tfidfVectorizer</a>.  This method counts all the occurrences of a given word and normalizes the counts by the Euclidean length of the vector of word occurrence counts (L<sup>2</sup>), this value is the tf in tf-idf.  The idf, or inverse document frequency is the log of all the documents divided by the count of the documents containing the word of interest.  Multiplying by the idf helps account for words that appear in all recipes, if a word appears in all the recipes its idf will tend toward zero.')
    w.write('</body><br><br>')
    for line in lines[2:-1]:
        w.write(line)
    w.write('\n{% endblock %}')

In [342]:
tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(1,1),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Ingredients'])
out = tfidf.transform(df['Ingredients']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Single Words','TFIDF Score']
chart1 = alt.Chart(values).mark_bar().encode(x=alt.X('TFIDF Score'),y=alt.Y('Single Words',sort=None))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(2,2),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Ingredients'])
out = tfidf.transform(df['Ingredients']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Bigrams','TFIDF Score']
chart2 = alt.Chart(values).mark_bar().encode(y=alt.Y('Bigrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(3,3),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Ingredients'])
out = tfidf.transform(df['Ingredients']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Trigrams','TFIDF Score']
chart3 = alt.Chart(values).mark_bar().encode(y=alt.Y('Trigrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

tfidf = TfidfVectorizer(min_df =1, max_features = 20,ngram_range=(4,4),stop_words=STOP_WORDS.union({'ll', 've','de','recipe'}))
tfidf = tfidf.fit(df['Ingredients'])
out = tfidf.transform(df['Ingredients']).toarray().sum(axis=0)
values = {x:out[tfidf.vocabulary_[x]] for x in tfidf.vocabulary_.keys()}
values = np.array(sorted([[x,float(y)] for x,y in values.items()],key=lambda x: -x[1]))
values = pd.DataFrame(values)
values[1] = values[1].astype('float')
values[0] = values[0].apply(lambda x: x[0].upper()+x[1:])
values.columns = ['Quadgrams','TFIDF Score']
chart4 = alt.Chart(values).mark_bar().encode(y=alt.Y('Quadgrams',sort=None),x=alt.X('TFIDF Score'))\
.properties(width = 500,height = 500)

alt.vconcat((chart1 | chart2) , (chart3 | chart4)).configure_axis( labelFontSize=12).save('templates/ing-ngram.html')
with open('templates/ing-ngram.html','r') as f:
    lines = f.readlines()
with open('templates/ing-ngram.html','w') as w:
    w.write('{% extends "layout.html" %}\n{% block title %}Home{% endblock %}\n{% block content %}')
    w.write('\n')
    w.write('<br><br><br>')
    w.write('<body class="starter-template text-center py-2">')
    w.write('Utilizing natural language processing we can investigate common n-grams in the ingredients list across all recipes.  The term-frequency times the inverse document-frequency (tf-idf) is calculated in scikit learn using their <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">tfidfVectorizer</a>.  This method counts all the occurrences of a given word and normalizes the counts by the Euclidean length of the vector of word occurrence counts (L<sup>2</sup>), this value is the tf in tf-idf.  The idf, or inverse document frequency is the log of all the documents divided by the count of the documents containing the word of interest.  Multiplying by the idf helps account for words that appear in all recipes, if a word appears in all the recipes its idf will tend toward zero.')
    w.write('</body><br><br>')
    for line in lines[2:-1]:
        w.write(line)
    w.write('\n{% endblock %}')

In [11]:
length_df = df[df['Publish Date']<730]
length_df = length_df[['Original Recipe Website','lname','ldesc','lingredients','linstructions','Number of Ratings']]

In [12]:
def striphtml(html):
    try:
        web = re.findall('www\.([A-z]*)\.',html)
        return web[0]
    except:
        return None


In [13]:
length_df['web'] = length_df['Original Recipe Website'].apply(striphtml)
length_df = length_df[['web','lname','ldesc','lingredients','linstructions','Number of Ratings']]

In [14]:
def dropout(x,amax):
    if x < amax:
        return x
    else:
        return amax

In [16]:
length_df['ldesc'] = length_df['ldesc'].apply(lambda x: dropout(x,400))

In [19]:
length_df.columns = ['Website','Words in the Recipe Name','Words in the Description','Number of Ingredients','Number of Instructions','Number of Ratings']

In [50]:
input_dropdown = alt.binding_select(options=['allrecipes','foodnetwork','epicurious'])
selection= alt.selection_multi(fields=['Website'],bind='legend')
color = alt.condition(selection,
                     alt.Color('Website:N'),
                     alt.value('lightgray'))
opacity = alt.condition(selection, alt.value(1.0),alt.value(0.0))
alt.data_transformers.disable_max_rows()
alt.Chart(length_df).mark_point().encode(
alt.X(alt.repeat("repeat"),type='quantitative'),
y='Number of Ratings:Q',
color=color,
opacity=opacity).add_selection(selection).repeat(repeat=['Words in the Recipe Name','Words in the Description','Number of Ingredients','Number of Instructions'],columns=2).interactive()\
.save('templates/lengths.html')

In [51]:
with open('templates/lengths.html','r') as f:
    lines = f.readlines()
with open('templates/lengths.html','w') as w:
    w.write('{% extends "layout.html" %}\n{% block title %}Home{% endblock %}\n{% block content %}')
    w.write('\n')
    w.write('<br><br><br>')
    w.write('<body class="starter-template text-center py-2">')
    w.write('Plots of the length of different sections of the recipe vs the number of reviews a recipe recieves.  This data is for the past two years of recipes on <a href="https://www.allrecipes.com">allrecipes</a>, <a href="https://www.foodnetwork.com">The Food Network</a>, and <a href="https://www.epicurious.com">Epicurious</a>.')
    w.write('</body><br><br>')
    for line in lines[2:-1]:
        w.write(line)
    w.write('\n{% endblock %}')

In [371]:
length_df.web.unique()

array(['allrecipes', 'epicurious', 'foodnetwork'], dtype=object)